In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
OLLAMA_API="http://localhost:11434/api/chat"
HEADERS={"Content-Type":"application/json"}
MODEL="llama3.2"


In [13]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        self.url=url
        response=requests.get(url, headers=headers)
        soup=BeautifulSoup(response.content, 'html.parser')
        self.title=soup.title.string if soup.title else 'No title found'
        for irr in soup.body(['Script', 'style', 'img','input']):
            irr.decompose()
        self.text=soup.body.get_text(separator='\n' ,strip=True)

In [32]:
link=input('Provide website link: ')
web=Website(f'{link}')


KeyboardInterrupt: Interrupted by user

In [21]:
system_prompt='You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown.'

In [22]:
def user_prompt_for(website):
    user_prompt=f'You are looking at webpage titled {website}'
    user_prompt+='\nThe contents of this website is as follows; \
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n'
    user_prompt+=website.text
    return user_prompt


In [ ]:
def message_for(website):
    return [
    {"role":"system", "content":f"{system_prompt}"},
    {"role":"user","content":f"{user_prompt_for(web)}"}
    ]

In [27]:
payload={
    "model": MODEL,
    "messages":message_for(web),
    "stream": False
}

In [ ]:
from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=message_for(web)
)

print(display(Markdown(response.choices[0].message.content)))